## Imports

In [102]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, classification_report

## Data Load

In [3]:
df_math = pd.read_csv("C:/Users/thoma/Tiago_Project/student-mat.csv")
df_port = pd.read_csv("C:/Users/thoma/Tiago_Project/student-por.csv")

print(df_math.shape)
print(df_port.shape)

(395, 33)
(649, 33)


## Sobre o Dataset

- school: Escola do aluno.
- sex: Sexo do aluno (ex: "F" - feminino ou "M" - masculino).
- age: Idade do aluno.
- address: Tipo de endereço de residência do aluno (ex: "U" - urbano ou "R" - rural).
- famsize: Tamanho da família (ex: "LE3" - menor ou igual a 3 ou "GT3" - maior que 3).
- Pstatus: Status de coabitação dos pais (ex: "T" - vivendo juntos ou "A" - separados).
- Medu: Nível de educação da mãe (0 - nenhum, 1 - educação primária, 2 – 5º a 9º ano, 3 – ensino médio ou 4 – ensino superior).
- Fedu: Nível de educação do pai (mesma escala que Medu).
- Mjob: Trabalho da mãe (ex: "teacher", "health", "services", "at_home", "other").
- Fjob: Trabalho do pai (mesma categorização que Mjob).
- reason: Razão para escolher a escola (ex: "home" - próximo de casa, "reputation" - reputação da escola, "course" - preferência de curso, "other" - outro motivo).
- guardian: Guardião do aluno (ex: "mother", "father", "other").
- traveltime: Tempo de viagem de casa para a escola (1 - <15 min., 2 - 15 a 30 min., 3 - 30 min. a 1 hora, 4 - >1 hora).
- studytime: Tempo semanal de estudo (1 - <2 horas, 2 - 2 a 5 horas, 3 - 5 a 10 horas, 4 - >10 horas).
- failures: Número de reprovações passadas (n se 1<=n<3, caso contrário 4).
- schoolsup: Suporte educacional extra (ex: "yes" ou "no").
- famsup: Suporte educacional da família (ex: "yes" ou "no").
- paid: Aulas particulares pagas (ex: "yes" ou "no"). activities: Atividades extracurriculares (ex: "yes" ou "no").
- nursery: Frequentou creche (ex: "yes" ou "no").
- higher: Deseja cursar ensino superior (ex: "yes" ou "no").
- internet: Acesso à Internet em casa (ex: "yes" ou "no").
- romantic: Em um relacionamento romântico (ex: "yes" ou "no").
- famrel: Qualidade das relações familiares (1 - muito ruim a 5 - excelente).
- freetime: Tempo livre após a escola (1 - muito baixo a 5 - muito alto).
- goout: Sair com amigos (1 - muito raro a 5 - muito frequente).
- Dalc: Consumo de álcool em dias de semana (1 - muito baixo a 5 - muito alto).
- Walc: Consumo de álcool nos fins de semana (1 - muito baixo a 5 - muito alto).
- health: Estado de saúde atual (1 - muito ruim a 5 - muito bom).
- absences: Número de ausências escolares.
- G1: Nota do primeiro período (numérico: de 0 a 20).
- G2: Nota do segundo período (numérico: de 0 a 20).
- G3: Nota final (numérico: de 0 a 20, output target).

## Fazendo o Merge dos Dataframes

In [4]:
# Colunas que serão usadas como chave pro merge
keys_merge = [
        'school', 'sex', 'age', 'address', 'famsize', 'Pstatus', 'Medu',
        'Fedu', 'Mjob', 'Fjob', 'reason', 'nursery', 'internet'
    ]

# Fazendo o merge utilizando as colunas definidas acima criando um sufixo nas colunas duplicadas
df_merge = df_math.merge(df_port, on=keys_merge, suffixes=('math', 'port'))

# Cria uma lista com as colunas que estão duplicadas e que contém o sufixo
lista_colunas_sufixo = [coluna for coluna in df_merge.columns if 'math' in coluna or 'port' in coluna]

# Remove as colunas duplicadas, deixando apenas colunas únicas e remove os sufixos
colunas_unicas = df_merge[lista_colunas_sufixo].columns.str.replace('math', '').str.replace('port', '').drop_duplicates()

# Remove as úktimas 3 colunas da lista que representam as notas e que são diferentes nos dois datasets
colunas_unicas = colunas_unicas.drop(colunas_unicas[-3:])

# Percorre o DF "colunas_unicas" agrupando pares de colunas com nomes iguais e com sufixo math e port
pares_colunas = [(coluna + 'math', coluna + 'port') for coluna in colunas_unicas]

# Mescla todos os pares de colunas
for col_mat, col_por in pares_colunas:
        df_merge[col_mat] = df_merge[col_mat].where(df_merge[col_mat] == df_merge[col_por])

# Remove as colunas que foram usadas para mesclar após a mesclagem, mantendo apenas as colunas originais do DataFrame da esquerda.
for _, col_por in pares_colunas:
        df_merge.drop(columns=[col_por], inplace=True)

# Remove colunas desnecessárias
df_merge = df_merge.drop(columns=['absencesmath', 'G1math', 'G2math', 'G1port', 'G2port'])

# Listas de colunas removidas e renomeadas
colunas_para_excluir = ['G3math', 'G3port']
colunas_para_renomear = [coluna for coluna in df_merge.columns if coluna not in colunas_para_excluir]


# Renomear colunas
for coluna in colunas_para_renomear:
        if coluna.endswith('math'):
            novo_nome_coluna = coluna.replace('math', '')
            df_merge.rename(columns={coluna: novo_nome_coluna}, inplace=True)

In [5]:
df_merge.head()

,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,...,internet,romantic,famrel,freetime,goout,Dalc,Walc,health,G3math,G3port
0,GP,F,18,U,GT3,A,4,4,at_home,teacher,...,no,no,4.0,3.0,4.0,1.0,1.0,3.0,6,11
1,GP,F,17,U,GT3,T,1,1,at_home,other,...,yes,no,5.0,3.0,3.0,1.0,1.0,3.0,6,11
2,GP,F,15,U,LE3,T,1,1,at_home,other,...,yes,no,4.0,3.0,2.0,2.0,3.0,3.0,10,12
3,GP,F,15,U,GT3,T,4,2,health,services,...,yes,yes,3.0,2.0,2.0,1.0,1.0,5.0,15,14
4,GP,F,16,U,GT3,T,3,3,other,other,...,no,no,4.0,3.0,2.0,1.0,2.0,5.0,10,13


In [6]:
df_merge.columns

Index(['school', 'sex', 'age', 'address', 'famsize', 'Pstatus', 'Medu', 'Fedu',
       'Mjob', 'Fjob', 'reason', 'guardian', 'traveltime', 'studytime',
       'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery',
       'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc',
       'Walc', 'health', 'G3math', 'G3port'],
      dtype='object')

## Data Cleaning

In [7]:
df = df_merge.copy()

In [8]:
# Identificando se há valores nulos e em quais colunas
df.isnull().sum()

school          0
sex             0
age             0
address         0
famsize         0
Pstatus         0
Medu            0
Fedu            0
Mjob            0
Fjob            0
reason          0
guardian        6
traveltime      5
studytime       9
failures       53
schoolsup       3
famsup          5
paid          171
activities      5
nursery         0
higher          4
internet        0
romantic        6
famrel         10
freetime       11
goout          10
Dalc            5
Walc           12
health          8
G3math          0
G3port          0
dtype: int64

In [9]:
# Neste caso, a solução proposta para tratar valores ausentes é substituir valores nulos pela moda. A moda é o valor que mais se repete na coluna.
df = df.fillna(df.mode().iloc[0])
df.isnull().sum()

school        0
sex           0
age           0
address       0
famsize       0
Pstatus       0
Medu          0
Fedu          0
Mjob          0
Fjob          0
reason        0
guardian      0
traveltime    0
studytime     0
failures      0
schoolsup     0
famsup        0
paid          0
activities    0
nursery       0
higher        0
internet      0
romantic      0
famrel        0
freetime      0
goout         0
Dalc          0
Walc          0
health        0
G3math        0
G3port        0
dtype: int64

In [10]:
df.head()

,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,...,internet,romantic,famrel,freetime,goout,Dalc,Walc,health,G3math,G3port
0,GP,F,18,U,GT3,A,4,4,at_home,teacher,...,no,no,4.0,3.0,4.0,1.0,1.0,3.0,6,11
1,GP,F,17,U,GT3,T,1,1,at_home,other,...,yes,no,5.0,3.0,3.0,1.0,1.0,3.0,6,11
2,GP,F,15,U,LE3,T,1,1,at_home,other,...,yes,no,4.0,3.0,2.0,2.0,3.0,3.0,10,12
3,GP,F,15,U,GT3,T,4,2,health,services,...,yes,yes,3.0,2.0,2.0,1.0,1.0,5.0,15,14
4,GP,F,16,U,GT3,T,3,3,other,other,...,no,no,4.0,3.0,2.0,1.0,2.0,5.0,10,13


## Data Preprocessing pipeline

In [11]:
df1=df.copy()

In [64]:
# Separando o dataset entre variáveis previsoras e vairáveis alvo.
x = df1.drop(columns=['G3math', "G3port"]) #Variáveis previsoras
y1 = df1['G3math'] #Variável Alvo
y2 = df1['G3port'] #Variável Alvo

In [40]:
x.head()

,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,...,nursery,higher,internet,romantic,famrel,freetime,goout,Dalc,Walc,health
0,GP,F,18,U,GT3,A,4,4,at_home,teacher,...,yes,yes,no,no,4.0,3.0,4.0,1.0,1.0,3.0
1,GP,F,17,U,GT3,T,1,1,at_home,other,...,no,yes,yes,no,5.0,3.0,3.0,1.0,1.0,3.0
2,GP,F,15,U,LE3,T,1,1,at_home,other,...,yes,yes,yes,no,4.0,3.0,2.0,2.0,3.0,3.0
3,GP,F,15,U,GT3,T,4,2,health,services,...,yes,yes,yes,yes,3.0,2.0,2.0,1.0,1.0,5.0
4,GP,F,16,U,GT3,T,3,3,other,other,...,yes,yes,no,no,4.0,3.0,2.0,1.0,2.0,5.0


In [52]:
# Realizando o Pre-processamento nas variáveis previsoras
# Separando as colunas ordinais, categóricas e numéricas.
ordinal_cols = [
    'Medu', 'Fedu', 'traveltime', 'studytime', 'failures', 'famrel',
    'freetime', 'goout', 'Dalc', 'Walc', 'health'
]
categorical_cols = [
    'school', 'sex', 'address', 'famsize', 'Pstatus', 'schoolsup', 'famsup',
    'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'Mjob',
    'Fjob', 'reason', 'guardian'
]

numerical_cols = ['age']

# Criando transformadores para cada tipo de coluna
ordinal_transformer = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)
numerical_transformer = StandardScaler()

# Criando um ColumnTransformer para aplicar transformadores específicos a colunas específicas
preprocessor = ColumnTransformer(
    transformers=[
        ('ordinal', ordinal_transformer, ordinal_cols),
        ('categorical', ordinal_transformer, categorical_cols),  # Usando OrdinalEncoder para colunas categóricas
        ('numerical', numerical_transformer, numerical_cols)
    ])

# Aplicando o pré-processamento aos dados
x_processed = preprocessor.fit_transform(x)
pd.DataFrame(X_processed, columns=x.columns).head()

,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,...,nursery,higher,internet,romantic,famrel,freetime,goout,Dalc,Walc,health
0,4.0,4.0,1.0,1.0,0.0,3.0,2.0,3.0,0.0,0.0,...,0.0,1.0,1.0,0.0,0.0,0.0,4.0,0.0,1.0,1.206223
1,1.0,1.0,0.0,1.0,0.0,4.0,2.0,2.0,0.0,0.0,...,0.0,0.0,1.0,1.0,0.0,0.0,2.0,0.0,0.0,0.352932
2,1.0,1.0,0.0,1.0,0.0,3.0,2.0,1.0,1.0,2.0,...,0.0,1.0,1.0,1.0,0.0,0.0,2.0,2.0,1.0,-1.353650
3,4.0,2.0,0.0,2.0,0.0,2.0,1.0,1.0,0.0,0.0,...,1.0,1.0,1.0,1.0,1.0,1.0,3.0,1.0,1.0,-1.353650
4,3.0,3.0,0.0,1.0,0.0,3.0,2.0,1.0,0.0,1.0,...,0.0,1.0,1.0,0.0,0.0,2.0,2.0,1.0,0.0,-0.500359


In [66]:
# Realizando o Pre-processamento nas variáveis alvo
def convert_to_binary(note):
    return 1 if note >= 10 else 0

 
y1=y1.apply(convert_to_binary)
y2=y2.apply(convert_to_binary)

## Machine Learning Model

### Regressão Logística

##### Dados Matemática

In [117]:
# Divisão dos dados após a transformação
X_train, X_test, y_train_mat, y_test_mat = train_test_split(x_processed, y1, test_size=0.3, random_state=42)

print('MAT:')
print(X_train.shape, X_test.shape)
print(y_train_mat.shape, y_test_mat.shape)

# Modelo de Regressão Logística para y1
logreg_y1 = LogisticRegression()
logreg_y1.fit(X_train, y_train_mat)
y1_pred = logreg_y1.predict(X_test)

# Avaliação do modelo para y1
accuracy_y1 = accuracy_score(y_test_mat, y1_pred)
accuracy_logreg_mat = accuracy_y1 * 100

print("Resultados para y1 usando Regressão Logística:")
print(f"Accuracy: {accuracy_logreg_mat:.2f}%")


MAT:
(267, 29) (115, 29)
(267,) (115,)
Resultados para y1 usando Regressão Logística:
Accuracy: 73.04%


##### Dados Português

In [118]:
# Divisão dos dados após a transformação
X_train, X_test, y_train_por, y_test_por = train_test_split(x_processed, y2, test_size=0.3, random_state=42)

print('PORT:')
print(X_train.shape, X_test.shape)
print(y_train_por.shape, y_test_por.shape)

# Modelo de Regressão Logística para y2
logreg_y2 = LogisticRegression()
logreg_y2.fit(X_train, y_train_mat)
y2_pred = logreg_y2.predict(X_test)

# Avaliação do modelo para y2
accuracy_y2 = accuracy_score(y_test_por, y2_pred)
accuracy_logreg_port = accuracy_y2 * 100

print("Resultados para y2 usando Regressão Logística:")
print(f"Accuracy: {accuracy_logreg_port:.2f}%")

PORT:
(267, 29) (115, 29)
(267,) (115,)
Resultados para y2 usando Regressão Logística:
Accuracy: 86.96%


### SVC (Support Vector Classifier)

##### Dados Matemática

In [119]:
# Divisão dos dados após a transformação
X_train, X_test, y_train_mat, y_test_mat = train_test_split(x_processed, y1, test_size=0.3, random_state=42)

print('MAT:')
print(X_train.shape, X_test.shape)
print(y_train_mat.shape, y_test_mat.shape)

# Modelo de Regressão Logística para y1
svc_y1 = SVC()
svc_y1.fit(X_train, y_train_mat)
y1_pred = svc_y1.predict(X_test)

# Avaliação do modelo para y1
accuracy_y1 = accuracy_score(y_test_mat, y1_pred)
accuracy_svc_mat = accuracy_y1 * 100

print("Resultados para y1 usando SVC:")
print(f"Accuracy: {accuracy_svc_mat:.2f}%")

MAT:
(267, 29) (115, 29)
(267,) (115,)
Resultados para y1 usando SVC:
Accuracy: 71.30%


##### Dados Português

In [120]:
# Divisão dos dados após a transformação
X_train, X_test, y_train_por, y_test_por = train_test_split(x_processed, y2, test_size=0.3, random_state=42)

print('PORT:')
print(X_train.shape, X_test.shape)
print(y_train_por.shape, y_test_por.shape)

# Modelo de Regressão Logística para y2
svc_y2 = SVC()
svc_y2.fit(X_train, y_train_mat)
y2_pred = svc_y2.predict(X_test)

# Avaliação do modelo para y2
accuracy_y2 = accuracy_score(y_test_por, y2_pred)
accuracy_svc_port = accuracy_y2 * 100

print("Resultados para y2 usando SVC:")
print(f"Accuracy: {accuracy_svc_port:.2f}%")

PORT:
(267, 29) (115, 29)
(267,) (115,)
Resultados para y2 usando SVC:
Accuracy: 88.70%


### Decision Tree Classifier

##### Dados Matemática

In [121]:
# Divisão dos dados após a transformação
X_train, X_test, y_train_mat, y_test_mat = train_test_split(x_processed, y1, test_size=0.3, random_state=42)

print('MAT:')
print(X_train.shape, X_test.shape)
print(y_train_mat.shape, y_test_mat.shape)

# Modelo de Regressão Logística para y1
decision_tree_y1 = DecisionTreeClassifier()
decision_tree_y1.fit(X_train, y_train_mat)
y1_pred = decision_tree_y1.predict(X_test)

# Avaliação do modelo para y1
accuracy_y1 = accuracy_score(y_test_mat, y1_pred)
accuracy_decisiontree_mat = accuracy_y1 * 100

print("Resultados para y1 usando DecisionTreeClassifier:")
print(f"Accuracy: {accuracy_decisiontree_mat:.2f}%")

MAT:
(267, 29) (115, 29)
(267,) (115,)
Resultados para y1 usando DecisionTreeClassifier:
Accuracy: 60.87%


##### Dados Português

In [122]:
# Divisão dos dados após a transformação
X_train, X_test, y_train_por, y_test_por = train_test_split(x_processed, y2, test_size=0.3, random_state=42)

print('PORT:')
print(X_train.shape, X_test.shape)
print(y_train_por.shape, y_test_por.shape)

# Modelo de Regressão Logística para y2
decision_tree_y2 = DecisionTreeClassifier()
decision_tree_y2.fit(X_train, y_train_mat)
y2_pred = decision_tree_y2.predict(X_test)

# Avaliação do modelo para y2
accuracy_y2 = accuracy_score(y_test_por, y2_pred)
accuracy_decisiontree_port = accuracy_y2 * 100

print("Resultados para y2 usando DecisionTreeClassifier:")
print(f"Accuracy: {accuracy_decisiontree_port:.2f}%")

PORT:
(267, 29) (115, 29)
(267,) (115,)
Resultados para y2 usando DecisionTreeClassifier:
Accuracy: 66.96%


### Random Forest Classifier

##### Dados Matemática

In [123]:
# Divisão dos dados após a transformação
X_train, X_test, y_train_mat, y_test_mat = train_test_split(x_processed, y1, test_size=0.3, random_state=42)

print('MAT:')
print(X_train.shape, X_test.shape)
print(y_train_mat.shape, y_test_mat.shape)

# Modelo de Regressão Logística para y1
random_forest_y1 = RandomForestClassifier(n_estimators=80, random_state=42)
random_forest_y1.fit(X_train, y_train_mat)
y1_pred = random_forest_y1.predict(X_test)

# Avaliação do modelo para y1
accuracy_y1 = accuracy_score(y_test_mat, y1_pred)
accuracy_randomforest_mat = accuracy_y1 * 100

print("Resultados para y1 usando RandomForestClassifier:")
print(f"Accuracy: {accuracy_randomforest_mat:.2f}%")

MAT:
(267, 29) (115, 29)
(267,) (115,)
Resultados para y1 usando RandomForestClassifier:
Accuracy: 72.17%


##### Dados Português

In [124]:
# Divisão dos dados após a transformação
X_train, X_test, y_train_por, y_test_por = train_test_split(x_processed, y2, test_size=0.3, random_state=42)

print('PORT:')
print(X_train.shape, X_test.shape)
print(y_train_por.shape, y_test_por.shape)

# Modelo de Regressão Logística para y2
random_forest_y2 = RandomForestClassifier(n_estimators=80, random_state=42)
random_forest_y2.fit(X_train, y_train_mat)
y2_pred = random_forest_y2.predict(X_test)

# Avaliação do modelo para y2
accuracy_y2 = accuracy_score(y_test_por, y2_pred)
accuracy_randomforest_port = accuracy_y2 * 100

print("Resultados para y2 usando RandomForestClassifier:")
print(f"Accuracy: {accuracy_randomforest_port:.2f}%")

PORT:
(267, 29) (115, 29)
(267,) (115,)
Resultados para y2 usando RandomForestClassifier:
Accuracy: 84.35%


### XGBoost

##### Dados Matemática

In [125]:
# Divisão dos dados após a transformação
X_train, X_test, y_train_mat, y_test_mat = train_test_split(x_processed, y1, test_size=0.3, random_state=42)

print('MAT:')
print(X_train.shape, X_test.shape)
print(y_train_mat.shape, y_test_mat.shape)

# Modelo de Regressão Logística para y1
xgboost_y1 = XGBClassifier()
xgboost_y1.fit(X_train, y_train_mat)
y1_pred = xgboost_y1.predict(X_test)

# Avaliação do modelo para y1
accuracy_y1 = accuracy_score(y_test_mat, y1_pred)
accuracy_xgboost_mat = accuracy_y1 * 100

print("Resultados para y1 usando XGBClassifier:")
print(f"Accuracy: {accuracy_xgboost_mat:.2f}%")

MAT:
(267, 29) (115, 29)
(267,) (115,)
Resultados para y1 usando XGBClassifier:
Accuracy: 63.48%


C:\Users\thoma\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\xgboost\data.py:520: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(data):


##### Dados Português

In [126]:
# Divisão dos dados após a transformação
X_train, X_test, y_train_por, y_test_por = train_test_split(x_processed, y2, test_size=0.3, random_state=42)

print('PORT:')
print(X_train.shape, X_test.shape)
print(y_train_por.shape, y_test_por.shape)

# Modelo de Regressão Logística para y2
xgboost_y2 = XGBClassifier()
xgboost_y2.fit(X_train, y_train_mat)
y2_pred = xgboost_y2.predict(X_test)

# Avaliação do modelo para y2
accuracy_y2 = accuracy_score(y_test_por, y2_pred)
accuracy_xgboost_port = accuracy_y2 * 100

print("Resultados para y2 usando XGBClassifier:")
print(f"Accuracy: {accuracy_xgboost_port:.2f}%")

PORT:
(267, 29) (115, 29)
(267,) (115,)
Resultados para y2 usando XGBClassifier:
Accuracy: 75.65%


C:\Users\thoma\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\xgboost\data.py:520: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(data):


## Resultados Finais

In [130]:
results = []

results.append(('Logistic Regression', 'Matemática', accuracy_logreg_mat))
results.append(('Logistic Regression', 'Português', accuracy_logreg_port))

results.append(('SVC', 'Matemática', accuracy_svc_mat))
results.append(('SVC', 'Português', accuracy_svc_port))

results.append(('Decision Tree Classifier', 'Matemática', accuracy_decisiontree_mat))
results.append(('Decision Tree Classifier', 'Português', accuracy_decisiontree_port))

results.append(('Random Forest Classifier', 'Matemática', accuracy_randomforest_mat))
results.append(('Random Forest Classifier', 'Português', accuracy_randomforest_port))

results.append(('XGBoost', 'Matemática', accuracy_xgboost_mat))
results.append(('XGBoost', 'Português', accuracy_xgboost_port))

final_results = pd.DataFrame(results, columns=['Modelo', 'Matéria', 'Acurácia'])
final_results.head(10)


,Modelo,Matéria,Acurácia
0,Logistic Regression,Matemática,73.043478
1,Logistic Regression,Português,86.956522
2,SVC,Matemática,71.304348
3,SVC,Português,88.695652
4,Decision Tree Classifier,Matemática,60.869565
5,Decision Tree Classifier,Português,66.956522
6,Random Forest Classifier,Matemática,72.173913
7,Random Forest Classifier,Português,84.347826
8,XGBoost,Matemática,63.478261
9,XGBoost,Português,75.652174


In [137]:
# Adiciona as colunas de previsão ao df_merge
df_merge['G3math_pred'] = logreg_y1.predict(x_processed)
df_merge['G3port_pred'] = svc_y2.predict(x_processed)

# Define uma função para transformar as previsões em 1 (aprovado) ou 0 (reprovado)
def transforma_aprovado_reprovado(valor):
    return 'Aprovado' if valor == 1 else 'Reprovado'

# Aplica a função à coluna 'G3math_pred'
df_merge['G3math_pred'] = df_merge['G3math_pred'].apply(transforma_aprovado_reprovado)
# Aplica a função à coluna 'G3port_pred'
df_merge['G3port_pred'] = df_merge['G3port_pred'].apply(transforma_aprovado_reprovado)

# Exporta o DataFrame para Excel
df_merge.to_excel("C:/Users/thoma/Tiago_Project/merge_with_predictions.xlsx", index=False)

### Conclusão

Concluímos que, o melhor algortimo previsor para a matéria de Matemática é o Logistic Regression e para a matéria de Português é o SVC.